In [ ]:
from pyspark.sql import SparkSession
import sys
import os

sys.path.append('../src')

# Buat konfigurasi Hadoop di Windows -> pny feli, kalian komen aja ini di tempat kalian
os.environ['HADOOP_HOME'] = "C:\\spark-4.0.0-bin-hadoop3"
os.environ['PATH'] = os.environ['PATH'] + ";" + os.environ['HADOOP_HOME'] + "\\bin"

# Import functions for feature engineering
# from feature_demographics import aggregate_demographics 
from feature_transactions import aggregate_transactions
#from feature_logs import aggregate_logs

spark = SparkSession.builder \
    .appName("FeatureEngineering") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

In [2]:
# Load the datasets
train_df = spark.read.csv("../dataset/train_combined.csv", header=True, inferSchema=True)
# members_df = spark.read.csv("dataset/members_v3.csv", header=True, inferSchema=True)
transactions_df = spark.read.csv("../dataset/transactions_combined.csv", header=True, inferSchema=True)
# user_logs_df = spark.read.csv("dataset/user_logs_combined/user_logs_combined.csv", header=True, inferSchema=True)

In [3]:
#demo_features = aggregate_demographics(members_df) 
trans_features = aggregate_transactions(transactions_df) 
#log_features = aggregate_logs(user_logs_df)

In [ ]:
#demo_features.write.mode("overwrite").parquet("data/demographic_features.parquet")
trans_features.write.mode("overwrite").parquet("data/transaction_features.parquet")
#log_features.write.mode("overwrite").parquet("data/log_features.parquet")

In [ ]:
master_table = train_df.join(demo_features, "msno", "left") \
                       .join(trans_features, "msno", "left") \
                       .join(log_features, "msno", "left")
# isi nilai null setelah join (misal, pengguna tanpa log)
master_table = master_table.fillna(0)
master_table.cache()

In [ ]:
master_table.write.mode("overwrite").parquet("data/master_feature_table.parquet")